<a href="https://colab.research.google.com/github/favasmfsm/hackerearth_carnival_wars/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

import numpy as np
import pandas as pd
import datetime
import matplotlib as plt

In [464]:
import numpy as np
import pandas as pd
import datetime
import matplotlib as plt

In [465]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn import datasets, linear_model, metrics
from sklearn.feature_selection import RFE

In [466]:
train=pd.read_csv('train.csv')
test= pd.read_csv('test.csv')

In [467]:
train.head()

,Product_id,Stall_no,instock_date,Market_Category,Customer_name,Loyalty_customer,Product_Category,Grade,Demand,Discount_avail,charges_1,charges_2 (%),Minimum_price,Maximum_price,Selling_Price
0,BRAE2NF6JA5GUEXG,37.0,2015-08-22 18:36:12.000,2,Lillyann,Yes,Fashion,1,68,0.0,376.0,11.0,2983.0,4713.0,4185.947700
1,TUNE8SFB6RJN2HSD,38.0,2016-03-27 21:19:13.000,24,Klynn,Yes,Fashion,0,51,0.0,397.0,12.0,7495.0,10352.0,9271.490256
2,BRAEAR7WZPQGPBZU,9.0,2015-08-18 19:25:22.000,447,Ridge,Yes,Child_care,0,10,0.0,250.0,9.0,5752.0,7309.0,6785.701362
3,WATDZ2ZQ8JPDHCTJ,50.0,2016-03-28 21:53:01.000,23,Abran,Yes,Educational,2,48,0.0,144.0,13.0,5090.0,20814.0,13028.917824
4,JWSEBUKYQPMBZ3RK,7.0,2016-03-29 22:58:53.000,63,Dustyn,Yes,Repair,1,35,1.0,211.0,4.0,2430.0,9261.0,906.553935


In [468]:
train['instock_date']=pd.to_datetime(train['instock_date'])
train['year']=train['instock_date'].dt.year
train['month']=train['instock_date'].dt.month
train['quarter']= train['instock_date'].dt.quarter
train['day_of_week']=train['instock_date'].dt.dayofweek
train['day_of_year']=train['instock_date'].dt.dayofyear
train['hour']=train['instock_date'].dt.hour

In [469]:
train.head()

,Product_id,Stall_no,instock_date,Market_Category,Customer_name,Loyalty_customer,Product_Category,Grade,Demand,Discount_avail,charges_1,charges_2 (%),Minimum_price,Maximum_price,Selling_Price,year,month,quarter,day_of_week,day_of_year,hour
0,BRAE2NF6JA5GUEXG,37.0,2015-08-22 18:36:12,2,Lillyann,Yes,Fashion,1,68,0.0,376.0,11.0,2983.0,4713.0,4185.947700,2015,8,3,5,234,18
1,TUNE8SFB6RJN2HSD,38.0,2016-03-27 21:19:13,24,Klynn,Yes,Fashion,0,51,0.0,397.0,12.0,7495.0,10352.0,9271.490256,2016,3,1,6,87,21
2,BRAEAR7WZPQGPBZU,9.0,2015-08-18 19:25:22,447,Ridge,Yes,Child_care,0,10,0.0,250.0,9.0,5752.0,7309.0,6785.701362,2015,8,3,1,230,19
3,WATDZ2ZQ8JPDHCTJ,50.0,2016-03-28 21:53:01,23,Abran,Yes,Educational,2,48,0.0,144.0,13.0,5090.0,20814.0,13028.917824,2016,3,1,0,88,21
4,JWSEBUKYQPMBZ3RK,7.0,2016-03-29 22:58:53,63,Dustyn,Yes,Repair,1,35,1.0,211.0,4.0,2430.0,9261.0,906.553935,2016,3,1,1,89,22


In [470]:
test['instock_date']=pd.to_datetime(test['instock_date'])
test['year']=test['instock_date'].dt.year
test['month']=test['instock_date'].dt.month
test['quarter']= test['instock_date'].dt.quarter
test['day_of_week']=test['instock_date'].dt.dayofweek
test['day_of_year']=test['instock_date'].dt.dayofyear
test['hour']=test['instock_date'].dt.hour

In [471]:
test.head()

,Product_id,Stall_no,instock_date,Market_Category,Customer_name,Loyalty_customer,Product_Category,Grade,Demand,Discount_avail,charges_1,charges_2 (%),Minimum_price,Maximum_price,year,month,quarter,day_of_week,day_of_year,hour
0,SCHE4YSTDVPVZVXW,39.0,2016-01-13 07:45:08,205,Ivanka,No,Fashion,0,90,0,380.0,16.0,2576.0,3340,2016,1,1,2,13,7
1,ACCEGCATKHNRXUHW,49.0,2015-08-23 20:37:05,3,Isaak,Yes,Fashion,0,87,0,393.0,16.0,1202.0,1955,2015,8,3,6,235,20
2,NKCE6GJ5XVJDXNNZ,1.0,2015-11-14 18:12:39,183,Analiese,No,Technology,2,55,0,493.0,9.0,7175.0,15715,2015,11,4,5,318,18
3,NKCEB8BK3ZXDHDHM,8.0,2015-11-21 04:56:19,358,Rusty,Yes,Child_care,3,86,0,303.0,16.0,5404.0,13078,2015,11,4,5,325,4
4,TOPEFDXSAHRNPF94,33.0,2015-05-06 15:09:46,167,Eloise,No,Technology,0,27,0,567.0,16.0,4069.0,6244,2015,5,2,2,126,15


In [472]:
train.shape


(6368, 21)

In [473]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6368 entries, 0 to 6367
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Product_id        6368 non-null   object        
 1   Stall_no          6363 non-null   float64       
 2   instock_date      6368 non-null   datetime64[ns]
 3   Market_Category   6368 non-null   int64         
 4   Customer_name     6157 non-null   object        
 5   Loyalty_customer  6368 non-null   object        
 6   Product_Category  6368 non-null   object        
 7   Grade             6368 non-null   int64         
 8   Demand            6368 non-null   int64         
 9   Discount_avail    6330 non-null   float64       
 10  charges_1         6170 non-null   float64       
 11  charges_2 (%)     6163 non-null   float64       
 12  Minimum_price     6330 non-null   float64       
 13  Maximum_price     6025 non-null   float64       
 14  Selling_Price     6327 n

In [474]:
for col in train.columns:
  print(col, train[col].nunique())

Product_id 6368
Stall_no 50
instock_date 5350
Market_Category 248
Customer_name 5900
Loyalty_customer 2
Product_Category 10
Grade 4
Demand 100
Discount_avail 2
charges_1 487
charges_2 (%) 17
Minimum_price 4236
Maximum_price 4618
Selling_Price 6321
year 3
month 11
quarter 4
day_of_week 7
day_of_year 214
hour 24


In [475]:
train.isnull().sum()

Product_id            0
Stall_no              5
instock_date          0
Market_Category       0
Customer_name       211
Loyalty_customer      0
Product_Category      0
Grade                 0
Demand                0
Discount_avail       38
charges_1           198
charges_2 (%)       205
Minimum_price        38
Maximum_price       343
Selling_Price        41
year                  0
month                 0
quarter               0
day_of_week           0
day_of_year           0
hour                  0
dtype: int64

In [476]:
train= train[train['Selling_Price'].notna()]

In [477]:
train['Customer_name']=train['Customer_name'].fillna(train['Customer_name'].mode()[0])
train['Maximum_price']=train['Maximum_price'].fillna(train['Maximum_price'].mean())
train['Minimum_price']=train['Minimum_price'].fillna(train['Minimum_price'].mean())
train['charges_1']=train['charges_1'].fillna(train['charges_1'].mean())
train['charges_2 (%)']=train['charges_2 (%)'].fillna(train['charges_2 (%)'].mean())
train['Discount_avail']=train['Discount_avail'].fillna(train['Discount_avail'].mode()[0])
train['Stall_no']=train['Stall_no'].fillna(train['Stall_no'].median())

In [478]:
train.isnull().sum()

Product_id          0
Stall_no            0
instock_date        0
Market_Category     0
Customer_name       0
Loyalty_customer    0
Product_Category    0
Grade               0
Demand              0
Discount_avail      0
charges_1           0
charges_2 (%)       0
Minimum_price       0
Maximum_price       0
Selling_Price       0
year                0
month               0
quarter             0
day_of_week         0
day_of_year         0
hour                0
dtype: int64

In [479]:
train['charges_1(%)']=train['charges_1']/100
train['charges_2']=train['charges_2 (%)']*100
train['total_charges']=train['charges_1']+train['charges_2']
train['range']=train['Maximum_price']-train['Minimum_price']

In [480]:
train.head()

,Product_id,Stall_no,instock_date,Market_Category,Customer_name,Loyalty_customer,Product_Category,Grade,Demand,Discount_avail,charges_1,charges_2 (%),Minimum_price,Maximum_price,Selling_Price,year,month,quarter,day_of_week,day_of_year,hour,charges_1(%),charges_2,total_charges,range
0,BRAE2NF6JA5GUEXG,37.0,2015-08-22 18:36:12,2,Lillyann,Yes,Fashion,1,68,0.0,376.0,11.0,2983.0,4713.0,4185.947700,2015,8,3,5,234,18,3.76,1100.0,1476.0,1730.0
1,TUNE8SFB6RJN2HSD,38.0,2016-03-27 21:19:13,24,Klynn,Yes,Fashion,0,51,0.0,397.0,12.0,7495.0,10352.0,9271.490256,2016,3,1,6,87,21,3.97,1200.0,1597.0,2857.0
2,BRAEAR7WZPQGPBZU,9.0,2015-08-18 19:25:22,447,Ridge,Yes,Child_care,0,10,0.0,250.0,9.0,5752.0,7309.0,6785.701362,2015,8,3,1,230,19,2.50,900.0,1150.0,1557.0
3,WATDZ2ZQ8JPDHCTJ,50.0,2016-03-28 21:53:01,23,Abran,Yes,Educational,2,48,0.0,144.0,13.0,5090.0,20814.0,13028.917824,2016,3,1,0,88,21,1.44,1300.0,1444.0,15724.0
4,JWSEBUKYQPMBZ3RK,7.0,2016-03-29 22:58:53,63,Dustyn,Yes,Repair,1,35,1.0,211.0,4.0,2430.0,9261.0,906.553935,2016,3,1,1,89,22,2.11,400.0,611.0,6831.0


In [481]:
train=train.drop(['Product_id', 'instock_date'],axis=1 )
train.head()

,Stall_no,Market_Category,Customer_name,Loyalty_customer,Product_Category,Grade,Demand,Discount_avail,charges_1,charges_2 (%),Minimum_price,Maximum_price,Selling_Price,year,month,quarter,day_of_week,day_of_year,hour,charges_1(%),charges_2,total_charges,range
0,37.0,2,Lillyann,Yes,Fashion,1,68,0.0,376.0,11.0,2983.0,4713.0,4185.947700,2015,8,3,5,234,18,3.76,1100.0,1476.0,1730.0
1,38.0,24,Klynn,Yes,Fashion,0,51,0.0,397.0,12.0,7495.0,10352.0,9271.490256,2016,3,1,6,87,21,3.97,1200.0,1597.0,2857.0
2,9.0,447,Ridge,Yes,Child_care,0,10,0.0,250.0,9.0,5752.0,7309.0,6785.701362,2015,8,3,1,230,19,2.50,900.0,1150.0,1557.0
3,50.0,23,Abran,Yes,Educational,2,48,0.0,144.0,13.0,5090.0,20814.0,13028.917824,2016,3,1,0,88,21,1.44,1300.0,1444.0,15724.0
4,7.0,63,Dustyn,Yes,Repair,1,35,1.0,211.0,4.0,2430.0,9261.0,906.553935,2016,3,1,1,89,22,2.11,400.0,611.0,6831.0


In [482]:
X=train.drop(['Selling_Price'],axis=1)
y=train['Selling_Price']
#print(X.dtypes)
X=X.iloc[:,:]
y.head()
X.head()

,Stall_no,Market_Category,Customer_name,Loyalty_customer,Product_Category,Grade,Demand,Discount_avail,charges_1,charges_2 (%),Minimum_price,Maximum_price,year,month,quarter,day_of_week,day_of_year,hour,charges_1(%),charges_2,total_charges,range
0,37.0,2,Lillyann,Yes,Fashion,1,68,0.0,376.0,11.0,2983.0,4713.0,2015,8,3,5,234,18,3.76,1100.0,1476.0,1730.0
1,38.0,24,Klynn,Yes,Fashion,0,51,0.0,397.0,12.0,7495.0,10352.0,2016,3,1,6,87,21,3.97,1200.0,1597.0,2857.0
2,9.0,447,Ridge,Yes,Child_care,0,10,0.0,250.0,9.0,5752.0,7309.0,2015,8,3,1,230,19,2.50,900.0,1150.0,1557.0
3,50.0,23,Abran,Yes,Educational,2,48,0.0,144.0,13.0,5090.0,20814.0,2016,3,1,0,88,21,1.44,1300.0,1444.0,15724.0
4,7.0,63,Dustyn,Yes,Repair,1,35,1.0,211.0,4.0,2430.0,9261.0,2016,3,1,1,89,22,2.11,400.0,611.0,6831.0


In [483]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
le=LabelEncoder()
X['Loyalty_customer']=le.fit_transform(X['Loyalty_customer'])
#X['Product_Category']=le.fit_transform(X['Product_Category'])
X['Customer_name']=le.fit_transform(X['Customer_name'])
X



,Stall_no,Market_Category,Customer_name,Loyalty_customer,Product_Category,Grade,Demand,Discount_avail,charges_1,charges_2 (%),Minimum_price,Maximum_price,year,month,quarter,day_of_week,day_of_year,hour,charges_1(%),charges_2,total_charges,range
0,37.0,2,3706,1,Fashion,1,68,0.0,376.0,11.0,2983.0,4713.0,2015,8,3,5,234,18,3.76,1100.0,1476.0,1730.0
1,38.0,24,3427,1,Fashion,0,51,0.0,397.0,12.0,7495.0,10352.0,2016,3,1,6,87,21,3.97,1200.0,1597.0,2857.0
2,9.0,447,4790,1,Child_care,0,10,0.0,250.0,9.0,5752.0,7309.0,2015,8,3,1,230,19,2.50,900.0,1150.0,1557.0
3,50.0,23,67,1,Educational,2,48,0.0,144.0,13.0,5090.0,20814.0,2016,3,1,0,88,21,1.44,1300.0,1444.0,15724.0
4,7.0,63,1740,1,Repair,1,35,1.0,211.0,4.0,2430.0,9261.0,2016,3,1,1,89,22,2.11,400.0,611.0,6831.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6363,17.0,24,4030,1,Technology,0,47,0.0,614.0,10.0,8174.0,11266.0,2016,3,1,0,88,19,6.14,1000.0,1614.0,3092.0
6364,21.0,55,5386,1,Child_care,0,35,0.0,269.0,5.0,3564.0,4920.0,2015,2,1,2,42,15,2.69,500.0,769.0,1356.0
6365,36.0,358,2143,0,Repair,3,29,0.0,283.0,3.0,5303.0,14614.0,2015,11,4,3,323,17,2.83,300.0,583.0,9311.0
6366,27.0,452,4594,1,Repair,0,8,0.0,267.0,3.0,4334.0,5849.0,2015,12,4,1,342,4,2.67,300.0,567.0,1515.0


In [484]:
for col in train.columns:
  print(col, train[col].nunique())

Stall_no 50
Market_Category 248
Customer_name 5867
Loyalty_customer 2
Product_Category 10
Grade 4
Demand 100
Discount_avail 2
charges_1 487
charges_2 (%) 18
Minimum_price 4217
Maximum_price 4594
Selling_Price 6321
year 3
month 11
quarter 4
day_of_week 7
day_of_year 214
hour 24
charges_1(%) 487
charges_2 18
total_charges 1914
range 3938


In [485]:
ohe=OneHotEncoder(handle_unknown='ignore')
ohe_pc_df=pd.DataFrame(ohe.fit_transform(X[['Product_Category']]).toarray())
X = X.join(ohe_pc_df,rsuffix='Product_category_is:')


In [486]:
ohe_year_df=pd.DataFrame(ohe.fit_transform(X[['year']]).toarray())
X = X.join(ohe_year_df,rsuffix='year_is:')

In [487]:
X[:30]

,Stall_no,Market_Category,Customer_name,Loyalty_customer,Product_Category,Grade,Demand,Discount_avail,charges_1,charges_2 (%),Minimum_price,Maximum_price,year,month,quarter,day_of_week,day_of_year,hour,charges_1(%),charges_2,total_charges,range,0,1,2,3,4,5,6,7,8,9,0year_is:,1year_is:,2year_is:
0,37.0,2,3706,1,Fashion,1,68,0.0,376.000000,11.0,2983.0,4713.000000,2015,8,3,5,234,18,3.760000,1100.0,1476.000000,1730.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,38.0,24,3427,1,Fashion,0,51,0.0,397.000000,12.0,7495.0,10352.000000,2016,3,1,6,87,21,3.970000,1200.0,1597.000000,2857.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,9.0,447,4790,1,Child_care,0,10,0.0,250.000000,9.0,5752.0,7309.000000,2015,8,3,1,230,19,2.500000,900.0,1150.000000,1557.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,50.0,23,67,1,Educational,2,48,0.0,144.000000,13.0,5090.0,20814.000000,2016,3,1,0,88,21,1.440000,1300.0,1444.000000,15724.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,7.0,63,1740,1,Repair,1,35,1.0,211.000000,4.0,2430.0,9261.000000,2016,3,1,1,89,22,2.110000,400.0,611.000000,6831.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
5,23.0,162,1064,0,Technology,1,49,0.0,633.000000,6.0,3959.0,6104.000000,2015,9,3,4,268,16,6.330000,600.0,1233.000000,2145.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
6,33.0,462,5842,0,Cosmetics,3,57,0.0,263.000000,13.0,2006.0,5257.000000,2014,12,4,0,342,12,2.630000,1300.0,1563.000000,3251.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7,4.0,23,3432,1,Educational,1,67,0.0,104.000000,15.0,3596.0,5921.000000,2015,9,3,1,265,14,1.040000,1500.0,1604.000000,2325.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
8,26.0,156,5399,1,Child_care,0,53,0.0,310.000000,6.0,6965.0,10194.000000,2016,3,1,0,88,21,3.100000,600.0,910.000000,3229.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9,5.0,275,4413,1,Child_care,0,84,1.0,321.000000,8.0,1103.0,5430.000000,2015,8,3,2,231,22,3.210000,800.0,1121.000000,4327.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [488]:
X=X.drop(['Product_Category','year'],axis=1)

In [489]:
X

,Stall_no,Market_Category,Customer_name,Loyalty_customer,Grade,Demand,Discount_avail,charges_1,charges_2 (%),Minimum_price,Maximum_price,month,quarter,day_of_week,day_of_year,hour,charges_1(%),charges_2,total_charges,range,0,1,2,3,4,5,6,7,8,9,0year_is:,1year_is:,2year_is:
0,37.0,2,3706,1,1,68,0.0,376.0,11.0,2983.0,4713.0,8,3,5,234,18,3.76,1100.0,1476.0,1730.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,38.0,24,3427,1,0,51,0.0,397.0,12.0,7495.0,10352.0,3,1,6,87,21,3.97,1200.0,1597.0,2857.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,9.0,447,4790,1,0,10,0.0,250.0,9.0,5752.0,7309.0,8,3,1,230,19,2.50,900.0,1150.0,1557.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,50.0,23,67,1,2,48,0.0,144.0,13.0,5090.0,20814.0,3,1,0,88,21,1.44,1300.0,1444.0,15724.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,7.0,63,1740,1,1,35,1.0,211.0,4.0,2430.0,9261.0,3,1,1,89,22,2.11,400.0,611.0,6831.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6363,17.0,24,4030,1,0,47,0.0,614.0,10.0,8174.0,11266.0,3,1,0,88,19,6.14,1000.0,1614.0,3092.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6364,21.0,55,5386,1,0,35,0.0,269.0,5.0,3564.0,4920.0,2,1,2,42,15,2.69,500.0,769.0,1356.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6365,36.0,358,2143,0,3,29,0.0,283.0,3.0,5303.0,14614.0,11,4,3,323,17,2.83,300.0,583.0,9311.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6366,27.0,452,4594,1,0,8,0.0,267.0,3.0,4334.0,5849.0,12,4,1,342,4,2.67,300.0,567.0,1515.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Training XGBoost on the Training set
from xgboost import XGBClassifier
classifier = XGBClassifier()
classifier.fit(X_train, y_train)


In [ ]:
# Training XGBoost on the Training set
from xgboost import XGBClassifier
classifier = XGBClassifier()
classifier.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

# Applying k-Fold Cross Validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))